---

# Pseudo-random number generator from scratch

Author: Cynthia Xiong

Date: 07/07/2022

This notebook was created as part of a challenge in the MLH Global Hack Week: INIT (2023). The goal is to create a pseudo-random number generator without the random module or any built-in PRNGs.

---

In [ ]:
import numpy as np

#### Linear-feedback shift register (4-bit)

In [ ]:
# Setting the initial state
state = [0,1,1,0]

In [ ]:
# XOR the last two numbers
new_num = state[-1] ^ state[-2]

# Put the new number in the front
state = [new_num, *state[:-1]]

# Translate the state from binary to decimal
num = state[3] + (state[2]*2) + (state[1]*4) + (state[0]*8)

# This cell should loop after the 16th run
num

In [ ]:
def LFSR_4(state):
  '''
  This function uses a 4-bit linear-feedback shift register to generate pseudo-random numbers. It is very limited, having only
  16 different states.
  Inputs:
    state = An array of 0's and 1's with a length of 4.
  Outputs:
    num = A pseudo-random number between 1 and 16.
    state = An array of 0's and 1's with a length of 4. This should be fed back into the function to keep it pseudo-random.
  '''
  new_num = state[-1] ^ state[-2]
  state = [new_num, *state[:-1]]
  num = state[3] + (state[2]*2) + (state[1]*4) + (state[0]*8)
  return num, state

In [ ]:
# This cell can loop, too
num,state = LFSR_4(state)
num

In [ ]:
# Speaking of loops...
for i in range(16):
  num,state = LFSR_4(state)
  print(num, end=' ')

---

#### Linear-feedback shift register (64-bit)

In [ ]:
# Now let's try something bigger
# This should randomly return a 0 or 1
np.random.randint(2)

In [ ]:
# A for-loop to make the initial state, for convenience
state = []
for i in range(32):
  state = [np.random.randint(2),np.random.randint(2),*state]

len(state)

In [ ]:
# New number and changing the state from the 4-bit LFSR
new_num = state[-1] ^ state[-2]
state = [new_num, *state[:-1]]

# Translate the state from binary to decimal
num = 0
for i in range(len(state)): # This loop is technically reading the binary code backwards, but eh
  num += state[i]*(2**i) 

# This cell should loop for a lot more runs than the 4-bit version
num

In [ ]:
def LFSR_64(state):
  '''
  This function uses a 64-bit linear-feedback shift register to generate pseudo-random numbers.
  Inputs:
    state = An array of 0's and 1's with a length of 64.
  Outputs:
    num = A pseudo-random number between 1 and (2^64)-1.
    state = An array of 0's and 1's with a length of 64. This should be fed back into the function to keep it pseudo-random.
  '''
  new_num = state[-1] ^ state[-2]
  state = [new_num, *state[:-1]]
  num = 0
  for i in range(len(state)):
    num += state[i]*(2**i)
  return num, state

In [ ]:
# This should be the max:
x = 0
for i in range(64):
  x += 2**i
x

In [ ]:
num,state = LFSR_64(state)
num

---

#### Using the LSFR (64-bit) to get a pseudo-random number within a specified range

In [ ]:
# I'll be using the 4-bit LFSR for testing purposes
state_4 = [1,0,0,1]

In [ ]:
# Choose a lower and upper bound
min = 2
max = 16

# Get the number and state
num,state_4 = LFSR_4(state_4)

# Use the modulo for the maximum value to get a number below the max
new_num = num % (max - min + 1) # -min because min will be added again later, and +1 because 3 % 3 = 0, which will make the intended max impossible to get

# Add the minimum value to get a number above the min
new_num += min

# Once again, this cell should loop after the 16th run
new_num

In [ ]:
# Now let's see if this works:
min = 2
max = 16

for i in range(16):
  num,state_4 = LFSR_4(state_4)
  new_num = num % (max - min + 1)
  new_num += min
  print(new_num, end=' ')

In [ ]:
# Let's make a function, except using the 64-bit LFSR
def RNG(min, max, state):
  '''
  This function generates a pseudo-random number within a specified range using a 64-bit linear-feedback shift register.
  Inputs:
    min = The lower bound in a range of numbers.
    max = The upper bound in a range of numbers.
    state = An array of 0's and 1's with a length of 64.
  Outputs:
    new_num = A pseudo-random number between the specified range.
    state = An array of 0's and 1's with a length of 64. This should be fed back into the function to keep it pseudo-random.
  '''
  num,state = LFSR_64(state)
  new_num = num % (max - min + 1)
  new_num += min
  return new_num, state

In [ ]:
num,state = RNG(12, 107, state)
num

---

## Final Product

RNG() generates a pseudo-random number within a specified range. Run the below cells to utilize it!

In [ ]:
# Run this cell to instantiate the functions for the psuedo-random number generator
def LFSR_64(state):
  new_num = state[-1] ^ state[-2]
  state = [new_num, *state[:-1]]
  num = 0
  for i in range(len(state)):
    num += state[i]*(2**i)
  return num, state

def RNG(min, max, state):
  num,state = LFSR_64(state)
  new_num = num % (max - min + 1)
  new_num += min
  return new_num, state

In [ ]:
# Run this cell to set up the state variable
import numpy as np
state = []
for i in range(32):
  state = [np.random.randint(2),np.random.randint(2),*state]

In [ ]:
# Specify the lower and upper bound in the range
lower_bound = 0
upper_bound = 100

In [ ]:
# After specifying the range, run this cell to generate a pseudo-random number
random_num,state = RNG(lower_bound, upper_bound, state)
print('Result:', random_num)